In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(subscription_id="d4ad7261-832d-46b2-b093-22156001df5b",resource_group="aml-quickstarts-141190",name="quick-starts-ws-141190")
exp = Experiment(workspace = ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141190
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-141190


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4

def reqd_cluster():
    cluster = 'my-cluster'
    try:
        compute_cluster = ComputeTarget(workspace = ws, name = cluster)
        print('Found existing compute cluster!')
    except ComputeTargetException:
        config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',max_nodes = 4)
        compute_cluster = ComputeTarget.create(ws, cluster, config)
    compute_cluster.wait_for_completion(show_output = True)
    return compute_cluster

In [3]:
compute_cluster = reqd_cluster()

Found existing compute cluster!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,randint,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--max_iter" : choice(5,10,20,40),
        "--C": uniform(0.5,1.0)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1,delay_evaluation = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",entry_script = 'train.py',compute_target = compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 25,
                                    max_concurrent_runs = 4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdconfig = exp.submit(config = hyperdrive_config)

In [6]:
RunDetails(hdconfig).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
import joblib
# Get your best run and save the model from that run.

best_run = hdconfig.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_ce643523-37da-4bad-91ea-bd7bd93aa4e6_1

 Accuracy: 0.9072837632776934


In [13]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.7174481997177157', '--max_iter', '20']


In [14]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_30c08bcaaab8db3bdf7e46e7fa1925dec2c8f99b3e7ac923a8a7eab9abb48718_d.txt',
 'azureml-logs/65_job_prep-tvmps_30c08bcaaab8db3bdf7e46e7fa1925dec2c8f99b3e7ac923a8a7eab9abb48718_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_30c08bcaaab8db3bdf7e46e7fa1925dec2c8f99b3e7ac923a8a7eab9abb48718_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/103_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [15]:
model = best_run.register_model(model_name = 'best_model', model_path = './')

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path = url)

In [19]:
from train import clean_data

x, y = clean_data(ds)

In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = ds,
    label_column_name = 'y',
    n_cross_validations = 7,
    compute_target = compute_cluster,
    iterations = 5)

In [21]:
# Submit your automl run

experiment = Experiment(ws, "automl_exp")
run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on my-cluster with default configuration
Running on remote compute: my-cluster
Parent Run ID: AutoML_3a12f3ba-a633-4391-9902-444a42b6903b

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a false

In [25]:
# Retrieve and save your best automl model.

best_run = run.get_best_child()
best_run.get_details()

{'runId': 'AutoML_3a12f3ba-a633-4391-9902-444a42b6903b_3',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-23T08:09:23.36447Z',
 'endTimeUtc': '2021-03-23T08:11:01.59579Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':5,\'is_timeseries\':False,\'name\':\'automl_exp\',\'compute_target\':\'my-cluster\',\'subscription_id\':\'d4ad7261-832d-46b2-b093-22156001df5b\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_3a12f3ba-a633-4391-9902-444a42b6903b_3","experiment_name":"automl_exp","workspace_name":"quick-starts-ws-141190","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","r

In [ ]:
compute_cluster.delete()